<a href="https://colab.research.google.com/github/DiffSharp/DiffSharp/blob/dev/notebooks/debug/NativeCudaLoadLinux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important note:** You should always work on a duplicate of the course notebook. On the page you used to open this, tick the box next to the name of the notebook and click duplicate to easily create a new version of this notebook.

You will get errors each time you try to update your course repository if you don't do this, and your changes will end up being erased by the original course version.

# Debugging CUDA package problems on Linux using CUDA-enabled machines on Google CoLab

Google Colab offers free CUDA-enabled GPU machines which are very useful for debugging problems with CUDA packages on Linux.

THis notebook started with the investigations in https://github.com/xamarin/TorchSharp/issues/169 and is being kept for future times we need to investigate similar failures


### Investigate GLIB and GLIBCXX dependencies available on this system

One reason Linux native binaries fail to load is when they have been built on a later Linux system, e.g. built on Ubuntu 20.04 and you're trying to load on Ubuntu 18.04.  This can even happen 18.04 (Build VM) to 18.04 (CoLab).  One particular pair of dependencies is "GLIB" (libc) and "GLIBCXX" (libstdc++).  Failures for these cause messages like "GLIBCXX_3.4.14 missing" - this is a version symbol in the native binaries.

Do it's important to determine the maximum GLIB and GLIBCXX symbols available on both the build machine and the target machine.  Here's an example:

In [ ]:
# Investigate GLIB and GLIBCXX dependencies available on this system
!ldd --version
!/sbin/ldconfig -p | grep stdc++
!strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep LIBCXX


## Install .NET SDK

To run F# code on CoLab you need to install the .NET SDK on the CoLab VM:

In [ ]:
# Install dotnet
!wget https://packages.microsoft.com/config/ubuntu/18.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb && sudo dpkg -i packages-microsoft-prod.deb && sudo apt-get update && sudo apt-get install -y apt-transport-https && sudo apt-get update && sudo apt-get install -y dotnet-sdk-5.0


In [ ]:
!dotnet --version

## Restore packages (libtorch-cpu)

Native loading for TorchSHarp.dll ultimately wants to bind to P/Invoke library "LibTorchSharp" which in turn has a bunch of dependencies on `torch.dll` or `libtorch.so` plus a whole lot of other things.


First we want to acquire the packages we want to load:

In [ ]:
# Restore packages (libtorch-cpu)
!echo "printfn \"phase0\"" > foo.fsx
!echo "#r \"nuget: TorchSharp, 0.92.52515\"" >> foo.fsx
!echo "#r \"nuget: libtorch-cpu, 1.9.0.10\"" >> foo.fsx
!echo "printfn \"done\"" >> foo.fsx
!cat foo.fsx
!echo "-------"
!dotnet fsi foo.fsx


printfn "phase0"
#r "nuget: TorchSharp, 0.92.52515"
#r "nuget: libtorch-cpu, 1.9.0.10"
printfn "done"
-------
==phase0
=done
=

Next we look around the dependencies of libLibTorchSharp.

In [ ]:
# Look around packages and dependencies  (libtorch-cpu)
! /lib/x86_64-linux-gnu/libc.so.6 --version
!ls /root/.nuget/packages/libtorch-cpu/1.9.0.10/runtimes/linux-x64/native
!ls /root/.nuget/packages/torchsharp/0.92.52515/runtimes/linux-x64/native/
!echo LD_LIBRARY_PATH=$LD_LIBRARY_PATH
!ldd --version
!ldd  /root/.nuget/packages/torchsharp/0.92.52515/runtimes/linux-x64/native/libLibTorchSharp.so


Next we run some F# code by creating a script and calling `TorchSharp.Torch.IsCudaAvailable`

In [ ]:
# Run some code with workaround  (libtorch-cpu)
!echo "printfn \"phase0\"" > foo.fsx
!echo "#r \"nuget: TorchSharp, 0.92.52515\"" >> foo.fsx
!echo "#r \"nuget: libtorch-cpu, 1.9.0.10\"" >> foo.fsx
#!echo "printfn \"phase1\"" >> foo.fsx
#!echo "open System.Runtime.InteropServices" >> foo.fsx
#!echo "NativeLibrary.Load(\"/root/.nuget/packages/libtorch-cpu/1.9.0.10/runtimes/linux-x64/native/libtorch.so\") |> printfn \"%A\"" >> foo.fsx
!echo "printfn \"phase2\"" >> foo.fsx
!echo "TorchSharp.Torch.IsCudaAvailable() |> printfn \"%A\"" >> foo.fsx
!cat foo.fsx
!dotnet fsi foo.fsx

This should report "false" since we're using the LibTorch CPU binaries and they can't connect to the GPU resource.

## Restore packages (libtorch-cuda)

Next we do a similar process for libtorch-cuda-11.7-linux-x64.

> This will take a long time as the packages are huge

> It will be faster on the next iteration

In [ ]:
# Restore packages (libtorch-cuda)
!echo "printfn \"phase0\"" > foo.fsx
!echo "#i \"nuget: https://donsyme.pkgs.visualstudio.com/TorchSharp/_packaging/packages2%40Local/nuget/v3/index.json\"" >> foo.fsx
!echo "#r \"nuget: TorchSharp, 0.92.52515\";;" >> foo.fsx
!echo "#r \"nuget: libtorch-cuda-11.7-linux-x64, 1.9.0.10\";;" >> foo.fsx
!echo "printfn \"done\"" >> foo.fsx
!cat foo.fsx
!echo "-------"
!dotnet fsi foo.fsx


printfn "phase0"
#i "nuget: https://donsyme.pkgs.visualstudio.com/TorchSharp/_packaging/packages2%40Local/nuget/v3/index.json"
#r "nuget: TorchSharp, 0.92.52515";;
#r "nuget: libtorch-cuda-11.7-linux-x64, 1.9.0.10";;
printfn "done"
-------
==phase0
==done
=

Next we look around the dependencies of libLibTorchSharp.

In [ ]:
# Look around packages and dependencies  (libtorch-cuda)
! /lib/x86_64-linux-gnu/libc.so.6 --version
!ls /root/.nuget/packages/libtorch-cuda-11.7-linux-x64/1.9.0.10/
!echo LD_LIBRARY_PATH=$LD_LIBRARY_PATH
!ldd --version
!ls /root/.nuget/packages/torchsharp/0.92.52515/runtimes/linux-x64/native/
#!ldd  /root/.nuget/packages/torchsharp/0.92.52515/runtimes/linux-x64/native/libLibTorchSharp.so
!ls /root/.nuget/packages/torchsharp/0.92.52515/lib/net6.0/cuda-11.7/
!ldd /root/.nuget/packages/torchsharp/0.92.52515/lib/net6.0/cuda-11.7/libLibTorchSharp.so

buildTransitive
lib
libtorch-cuda-11.7-linux-x64.1.9.0.10.nupkg
libtorch-cuda-11.7-linux-x64.1.9.0.10.nupkg.sha512
libtorch-cuda-11.7-linux-x64.nuspec
LICENSE-LIBTORCH.txt
LD_LIBRARY_PATH=/usr/lib64-nvidia
ldd (Ubuntu GLIBC 2.27-3ubuntu1.2) 2.27
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Written by Roland McGrath and Ulrich Drepper.
libLibTorchSharp.so
libc10_cuda.so			  libnvToolsExt-24de1d56.so.1
libc10d_cuda_test.so		  libprocess_group_agent.so
libc10.so			  libpytorch_jni.so
libcaffe2_detectron_ops_gpu.so	  libshm.so
libcaffe2_module_test_dynamic.so  libtensorpipe_agent.so
libcaffe2_nvrtc.so		  libtorchbind_test.so
libcaffe2_observers.so		  libtorch_cpu.so
libcudart-6d56b25a.so.11.0	  libtorch_cuda_cpp.so
libfbjni.so			  libtorch_cuda_cu.so
libgomp-7c85b1e2.so.1		  libtorch_cuda.so
libjitbackend_test.so		  libtorch_global_deps

In [ ]:
# Run some code with workaround  (libtorch-cpu)
!echo "printfn \"phase0\"" > foo.fsx
!echo "#r \"nuget: TorchSharp, 0.92.52515\"" >> foo.fsx
!echo "#r \"nuget: libtorch-cuda-11.7-linux-x64, 1.9.0.10\"" >> foo.fsx
#!echo "printfn \"phase1\"" >> foo.fsx
#!echo "open System.Runtime.InteropServices" >> foo.fsx
#!echo "NativeLibrary.Load(\"/root/.nuget/packages/libtorch-cpu/1.9.0.7/runtimes/linux-x64/native/libtorch.so\") |> printfn \"%A\"" >> foo.fsx
!echo "printfn \"phase2\"" >> foo.fsx
!echo "TorchSharp.Torch.IsCudaAvailable() |> printfn \"%A\"" >> foo.fsx
!cat foo.fsx
!dotnet fsi foo.fsx

This should report "true" since we're using the LibTorch CPU binaries and they can't connect to the GPU resource.